# 1. Import Packages

In [1]:
import os
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import jaccard_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from time import perf_counter

from mlrose_local import mlrose

import warnings
warnings.filterwarnings("ignore")

# 2. Load and clean the datasets

Data are downloaded from Kaggle:<br>
nba games https://www.kaggle.com/nathanlauga/nba-games<br>
nba_games.csv : All games from 2004 season to last update with the date, teams and some details like number of points, etc.

In [2]:
# Change the path into your current working directory
os.chdir(r"C:\Users\13102\Desktop\CS7641 Machine Learning Randomized Optimization")

nba_games = pd.read_csv('nba_games.csv')

We will calculate the different between home team and away team in five aspects, including FG_PCT(field goals percentage), FT_PCT(field throws percentage), FG3_PCT(three-point field goals percentage),AST(assists) and REB(rebounds). HOME_TEAM_WINS represents that if home team wins, the value is 1 and if home team loses, the value is 0.

In [3]:
data1 = nba_games[['GAME_DATE_EST','HOME_TEAM_WINS',]]
pd.options.mode.chained_assignment = None

# Select the data for 2019-2020 season from 2019-10-4 to 2020-3-1
start_date='2019-10-4'
end_date='2020-3-1'

data1['GAME_DATE_EST'] = pd.to_datetime(data1['GAME_DATE_EST'])  
mask = (data1['GAME_DATE_EST'] >= start_date) & (data1['GAME_DATE_EST'] <= end_date)
data1 = data1.loc[mask]

# Drop useless columns
data1 = data1.reset_index().drop(columns=['index', 'GAME_DATE_EST'])

cols1 = ['FG_PCT','FT_PCT','FG3_PCT','AST','REB']
for col in cols1:
    data1[col+'_diff'] = nba_games[col+'_home'].sub(nba_games[col+'_away'], axis = 0)

# Change datatype from float32 into int64
X1 = np.array(data1.values[:,1:-1],dtype='int64')
Y1 = np.array(data1.values[:,0],dtype='int64')
data1.describe(include='all')

,HOME_TEAM_WINS,FG_PCT_diff,FT_PCT_diff,FG3_PCT_diff,AST_diff,REB_diff
count,965.00000,965.000000,965.000000,965.000000,965.000000,965.000000
mean,0.54715,0.011031,0.001882,0.005033,1.174093,0.839378
std,0.49803,0.078655,0.146005,0.131647,6.439211,9.418250
min,0.00000,-0.208000,-0.571000,-0.427000,-22.000000,-39.000000
25%,0.00000,-0.043000,-0.092000,-0.086000,-3.000000,-5.000000
50%,1.00000,0.009000,0.000000,0.003000,1.000000,1.000000
75%,1.00000,0.068000,0.094000,0.098000,5.000000,7.000000
max,1.00000,0.258000,0.571000,0.458000,23.000000,31.000000


# 3. Get Train and test datasets

In [4]:
np.random.seed(10)
X_train, X_test, y_train, y_test = train_test_split( X1, Y1, test_size=0.30, random_state=4)

# Normalize feature data
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test) # One hot encode target values
one_hot = OneHotEncoder()
y_train_hot = one_hot.fit_transform(y_train.reshape(-1, 1)).todense()
y_test_hot = one_hot.transform(y_test.reshape(-1, 1)).todense()

# 4. Run Neural Networks model

In this model, we will compare the fitting time for a neural networks model. Therefore, we only calculate the fitting time.

In [5]:
#Train Model using MLPClassifier function on Relu Activation Function

clf = MLPClassifier(hidden_layer_sizes=(9,9), activation = 'relu', max_iter=1000, solver = 'lbfgs' )
time_start = perf_counter()
clf.fit(X_train_scaled, y_train_hot)
fit_time = perf_counter() - time_start
print(f'Train fitting time for NN model = {fit_time}')

time_start = perf_counter()
yhat = clf.predict(X_test_scaled)
fit_time = perf_counter() - time_start
print (f'Test fitting time for NN model = {fit_time}')

# Evaulation
jaccard = jaccard_score(y_test_hot, yhat, average='micro')
print("jaccard index: ",jaccard)
print (classification_report(y_test_hot, yhat))

Train fitting time for NN model = 0.13168369999999996
Test fitting time for NN model = 0.0007014000000005183
jaccard index:  0.457286432160804
              precision    recall  f1-score   support

           0       0.57      0.46      0.51       122
           1       0.66      0.75      0.70       168

   micro avg       0.63      0.63      0.63       290
   macro avg       0.61      0.60      0.60       290
weighted avg       0.62      0.63      0.62       290
 samples avg       0.63      0.63      0.63       290



In [6]:
#Train Model using MLPClassifier function on Sigmoid Activation Function

clf_sig = MLPClassifier(hidden_layer_sizes=(9,9), activation = 'logistic', max_iter=1000, solver = 'lbfgs' )
time_start = perf_counter()
clf_sig.fit(X_train_scaled, y_train_hot)
fit_time = perf_counter() - time_start
print(f'Train fitting time for NN model = {fit_time}')

time_start = perf_counter()
yhat_sig = clf_sig.predict(X_test_scaled)
fit_time = perf_counter() - time_start
print (f'Test fitting time for NN model = {fit_time}')

# Evaulation
jaccard_sig = jaccard_score(y_test_hot, yhat_sig, average='micro')
print("jaccard index: ",jaccard)
print (classification_report(y_test_hot, yhat_sig))

Train fitting time for NN model = 1.0322408999999997
Test fitting time for NN model = 0.0010769999999995505
jaccard index:  0.457286432160804
              precision    recall  f1-score   support

           0       0.56      0.33      0.41       122
           1       0.62      0.81      0.70       168

   micro avg       0.61      0.61      0.61       290
   macro avg       0.59      0.57      0.56       290
weighted avg       0.60      0.61      0.58       290
 samples avg       0.61      0.61      0.61       290



# 5. Use randomized hill climbing for optimization

Activation Function Relu

In [7]:
np.random.seed(7)
clf_hill = mlrose.NeuralNetwork(hidden_nodes = [2], activation = 'relu', 
                                algorithm = 'random_hill_climb', 
                                max_iters=1000, bias = True, is_classifier = True, 
                                learning_rate = 0.5, early_stopping = True, clip_max = 5, 
                                max_attempts = 100)
time_start = perf_counter()
clf_hill.fit(X_train_scaled, y_train_hot)
fit_time = perf_counter() - time_start
print(f'Train fitting time for NN model = {fit_time}')

Train fitting time for NN model = 0.23200109999999974


In [8]:
#Predict Labels for test set and assess accuracy
time_start = perf_counter()
yhat_hill_test = clf_hill.predict(X_test_scaled)
fit_time = perf_counter() - time_start
test_accuracy1 = accuracy_score(y_test_hot, yhat_hill_test)
f1 = f1_score(y_test_hot, yhat_hill_test, average='weighted') 
jaccard1 = jaccard_score(y_test_hot, yhat_hill_test, average='micro')
print (f'Test fitting time for NN model = {fit_time}')
print (f'accuracy score = {test_accuracy1}')
print("f1 score: ", f1)
print("jaccard index: ",jaccard1)
print (classification_report(y_test_hot, yhat_hill_test))

Test fitting time for NN model = 0.0009589999999999321
accuracy score = 0.6275862068965518
f1 score:  0.6196865203761756
jaccard index:  0.457286432160804
              precision    recall  f1-score   support

           0       0.57      0.46      0.51       122
           1       0.66      0.75      0.70       168

   micro avg       0.63      0.63      0.63       290
   macro avg       0.61      0.60      0.60       290
weighted avg       0.62      0.63      0.62       290
 samples avg       0.63      0.63      0.63       290



Activation Function Sigmoid

In [9]:
np.random.seed(7)
clf_hill_sig = mlrose.NeuralNetwork(hidden_nodes = [2], activation = 'sigmoid', 
                                algorithm = 'random_hill_climb', 
                                max_iters=1000, bias = True, is_classifier = True, 
                                learning_rate = 0.5, early_stopping = True, clip_max = 5, 
                                max_attempts = 100)
time_start = perf_counter()
clf_hill_sig.fit(X_train_scaled, y_train_hot)
fit_time = perf_counter() - time_start
print(f'Train fitting time for NN model = {fit_time}')

Train fitting time for NN model = 0.5258365000000005


In [10]:
#Predict Labels for test set and assess accuracy
time_start = perf_counter()
yhat_hill_test_sig = clf_hill_sig.predict(X_test_scaled)
fit_time = perf_counter() - time_start
test_accuracy2 = accuracy_score(y_test_hot, yhat_hill_test_sig)
f2 = f1_score(y_test_hot, yhat_hill_test_sig, average='weighted') 
jaccard2 = jaccard_score(y_test_hot, yhat_hill_test_sig, average='micro')
print (f'Test fitting time for NN model = {fit_time}')
print (f'accuracy score = {test_accuracy2}')
print("f1 score: ", f2)
print("jaccard index: ",jaccard2)
print (classification_report(y_test_hot, yhat_hill_test_sig))

Test fitting time for NN model = 0.0006398999999994714
accuracy score = 0.6275862068965518
f1 score:  0.6196865203761756
jaccard index:  0.457286432160804
              precision    recall  f1-score   support

           0       0.57      0.46      0.51       122
           1       0.66      0.75      0.70       168

   micro avg       0.63      0.63      0.63       290
   macro avg       0.61      0.60      0.60       290
weighted avg       0.62      0.63      0.62       290
 samples avg       0.63      0.63      0.63       290



# 6. Use simulated annealing for optimization

Activation Function Relu

In [11]:
np.random.seed(7)
clf_sim = mlrose.NeuralNetwork(hidden_nodes = [2], activation = 'relu', 
                                algorithm = 'simulated_annealing', 
                                max_iters=1000, bias = True, is_classifier = True, 
                                learning_rate = 0.5, early_stopping = True, clip_max = 5, 
                                max_attempts = 100)
time_start = perf_counter()
clf_sim.fit(X_train_scaled, y_train_hot)
fit_time = perf_counter() - time_start
print(f'Train fitting time for NN model = {fit_time}')

Search ended with attempts>max_attempts (1000>1000)
Train fitting time for NN model = 2.3663499000000012


In [12]:
#Predict Labels for test set and assess accuracy
time_start = perf_counter()
yhat_sim_test = clf_sim.predict(X_test_scaled)
fit_time = perf_counter() - time_start
test_accuracy3 = accuracy_score(y_test_hot, yhat_sim_test)
f3 = f1_score(y_test_hot, yhat_sim_test, average='weighted') 
jaccard3 = jaccard_score(y_test_hot, yhat_sim_test, average='micro')
print (f'Test fitting time for NN model = {fit_time}')
print (f'test accuracy score = {test_accuracy3}')
print("f1 score: ", f3)
print("jaccard index: ",jaccard3)
print (classification_report(y_test_hot, yhat_sim_test))

Test fitting time for NN model = 0.0012954999999994499
test accuracy score = 0.4206896551724138
f1 score:  0.24914630063608975
jaccard index:  0.2663755458515284
              precision    recall  f1-score   support

           0       0.42      1.00      0.59       122
           1       0.00      0.00      0.00       168

   micro avg       0.42      0.42      0.42       290
   macro avg       0.21      0.50      0.30       290
weighted avg       0.18      0.42      0.25       290
 samples avg       0.42      0.42      0.42       290



Activation Function Sigmoid

In [13]:
#np.random.seed(7)
clf_sim_sig = mlrose.NeuralNetwork(hidden_nodes = [2], activation = 'sigmoid', 
                                algorithm = 'simulated_annealing', 
                                max_iters=1000, bias = True, is_classifier = True, 
                                learning_rate = 0.5, early_stopping = True, clip_max = 5, 
                                max_attempts = 100)
time_start = perf_counter()
clf_sim_sig.fit(X_train_scaled, y_train_hot)
fit_time = perf_counter() - time_start
print(f'Train fitting time for NN model = {fit_time}')

Search ended with attempts>max_attempts (1000>1000)
Train fitting time for NN model = 2.2413022


In [14]:
#Predict Labels for test set and assess accuracy
time_start = perf_counter()
yhat_sim_test_sig = clf_sim_sig.predict(X_test_scaled)
fit_time = perf_counter() - time_start
test_accuracy4 = accuracy_score(y_test_hot, yhat_sim_test_sig)
f4 = f1_score(y_test_hot, yhat_sim_test_sig, average='weighted') 
jaccard4 = jaccard_score(y_test_hot, yhat_sim_test_sig, average='micro')
print (f'Test fitting time for NN model = {fit_time}')
print (f'test accuracy score = {test_accuracy4}')
print("f1 score: ", f4)
print("jaccard index: ",jaccard4)
print (classification_report(y_test_hot, yhat_sim_test_sig))

Test fitting time for NN model = 0.0010329000000002253
test accuracy score = 0.6275862068965518
f1 score:  0.6196865203761756
jaccard index:  0.457286432160804
              precision    recall  f1-score   support

           0       0.57      0.46      0.51       122
           1       0.66      0.75      0.70       168

   micro avg       0.63      0.63      0.63       290
   macro avg       0.61      0.60      0.60       290
weighted avg       0.62      0.63      0.62       290
 samples avg       0.63      0.63      0.63       290



# 7. Use a genetic algorithm for optimization

Activation Function Relu

In [15]:
np.random.seed(7)
clf_sim = mlrose.NeuralNetwork(hidden_nodes = [2], activation = 'relu', 
                                algorithm = 'simulated_annealing', 
                                max_iters=1000, bias = True, is_classifier = True, 
                                learning_rate = 0.5, early_stopping = True, clip_max = 5, 
                                max_attempts = 100)
time_start = perf_counter()
clf_sim.fit(X_train_scaled, y_train_hot)
fit_time = perf_counter() - time_start
print(f'Train fitting time for NN model = {fit_time}')

Search ended with attempts>max_attempts (1000>1000)
Train fitting time for NN model = 2.2715651


In [16]:
#Predict Labels for test set and assess accuracy
time_start = perf_counter()
yhat_sim_test = clf_sim.predict(X_test_scaled)
fit_time = perf_counter() - time_start
test_accuracy3 = accuracy_score(y_test_hot, yhat_sim_test)
f3 = f1_score(y_test_hot, yhat_sim_test, average='weighted') 
jaccard3 = jaccard_score(y_test_hot, yhat_sim_test, average='micro')
print (f'Test fitting time for NN model = {fit_time}')
print (f'test accuracy score = {test_accuracy3}')
print("f1 score: ", f3)
print("jaccard index: ",jaccard3)
print (classification_report(y_test_hot, yhat_sim_test))

Test fitting time for NN model = 0.0007366000000015305
test accuracy score = 0.4206896551724138
f1 score:  0.24914630063608975
jaccard index:  0.2663755458515284
              precision    recall  f1-score   support

           0       0.42      1.00      0.59       122
           1       0.00      0.00      0.00       168

   micro avg       0.42      0.42      0.42       290
   macro avg       0.21      0.50      0.30       290
weighted avg       0.18      0.42      0.25       290
 samples avg       0.42      0.42      0.42       290



Activation Function Sigmoid

In [17]:
#np.random.seed(7)
clf_sim_sig = mlrose.NeuralNetwork(hidden_nodes = [2], activation = 'sigmoid', 
                                algorithm = 'simulated_annealing', 
                                max_iters=1000, bias = True, is_classifier = True, 
                                learning_rate = 0.5, early_stopping = True, clip_max = 5, 
                                max_attempts = 100)
time_start = perf_counter()
clf_sim_sig.fit(X_train_scaled, y_train_hot)
fit_time = perf_counter() - time_start
print(f'Train fitting time for NN model = {fit_time}')

Search ended with attempts>max_attempts (1000>1000)
Train fitting time for NN model = 2.2230119000000013


In [18]:
#Predict Labels for test set and assess accuracy
time_start = perf_counter()
yhat_sim_test_sig = clf_sim_sig.predict(X_test_scaled)
fit_time = perf_counter() - time_start
test_accuracy4 = accuracy_score(y_test_hot, yhat_sim_test_sig)
f4 = f1_score(y_test_hot, yhat_sim_test_sig, average='weighted') 
jaccard4 = jaccard_score(y_test_hot, yhat_sim_test_sig, average='micro')
print (f'Test fitting time for NN model = {fit_time}')
print (f'test accuracy score = {test_accuracy4}')
print("f1 score: ", f4)
print("jaccard index: ",jaccard4)
print (classification_report(y_test_hot, yhat_sim_test_sig))

Test fitting time for NN model = 0.0007601000000008185
test accuracy score = 0.6275862068965518
f1 score:  0.6196865203761756
jaccard index:  0.457286432160804
              precision    recall  f1-score   support

           0       0.57      0.46      0.51       122
           1       0.66      0.75      0.70       168

   micro avg       0.63      0.63      0.63       290
   macro avg       0.61      0.60      0.60       290
weighted avg       0.62      0.63      0.62       290
 samples avg       0.63      0.63      0.63       290

